Read the updated runoff data from Louise (now also updated through fall 2022)

- Extract for SS and HB

In [1]:
from oyv.imports import *
import pandas as pd
import glob
from oyv.map import quickmap


In [2]:
pretty.rc()

In [3]:
figdir = '/home/oyvindl/work/projects/barents_glaciers/figures/runoff_mod/'

In [13]:
#ddir = '/home/oyvindl/work/projects/barents_glaciers/data/runoff_model/most_recent_louise/Avrenning_Austfonna_2022/'
ddir = '/home/oyvindl/work/projects/barents_glaciers/glacierocean-austfonna/data/runoff/data/runoff_model_data_louise/'
fns = glob.glob(ddir + '*.dat')

#### Units


From Louise: Units are **GT/day**

I want **m3/sec**. Assuming 1000 kg/m3, this is equivalent to **tons/sec**




In [14]:
fact_m3_day =1e9/86400 # Double check - assuming 1000 kg/m3 

In [15]:
A = pd.read_csv(fns[0], sep = ',')
ARR = A.to_numpy()

In [16]:
ca()

In [17]:
nns_ss = [513, 518, 522, 524, 525, 526] # 
nns_hb = [521, 520] # 
nns_hbss = nns_ss + nns_hb

In [18]:
#if False:# mqap of locations
if True:
    fig, ax = plt.subplots()
    m = quickmap.map_stere(3, 2, ax = ax, oc_col = 'w', lat0=79.7, lon0 =23, res = 'i')
    m.plot(A.Longitude.to_numpy(), A.Latitude.to_numpy(), '.k', latlon = True)

    for nn in nns_ss:
        m.plot(A.Longitude.to_numpy()[nn], A.Latitude.to_numpy()[nn], '*r', 
           ms = 10, latlon = True, label = nn)
    for nn in nns_hb:
        m.plot(A.Longitude.to_numpy()[nn], A.Latitude.to_numpy()[nn], '*b', 
           ms = 10, latlon = True, label = nn)
    ax.legend()

In [19]:
B = Bunch()
B.lon, B.lat = A.Longitude, A.Latitude 

for key in ['t', 'runoff_hb', 'runoff_ss', 'runoff_hbss', 'runoff_all']:
    B[key] = np.ma.array([])

B.sum_FWC_all = np.zeros(ARR.shape[0])

for yr in np.arange(2014, 2023):
    fn_yr = 'runoff_Austfonna_%i_k_0.9.dat'%yr
    A = pd.read_csv(ddir + fn_yr, sep = ',')
    ARR = A.to_numpy()
    
    ndays = ARR.shape[1]-4
    B.t = np.concatenate([B.t, num2date(date2num(dt.date(yr, 1, 1)) +
                                        np.arange(ndays)+0.5)])
    B.runoff_hb = np.concatenate([B.runoff_hb, ARR[nns_hb, 4:].sum(axis = 0)])
    B.runoff_ss = np.concatenate([B.runoff_ss, ARR[nns_ss, 4:].sum(axis = 0)])
    B.runoff_hbss = np.concatenate([B.runoff_hbss, ARR[nns_hbss, 4:].sum(axis = 0)])
    B.runoff_all = np.concatenate([B.runoff_all,  
                                   ARR[:, 4:].sum(axis = 0)])
    B.sum_FWC_all += ARR[:, 4:].sum(axis = 1)
    

In [20]:
ca()

In [21]:
fig, ax = plt.subplots(3, 1, sharex = True, figsize = (10, 8))

plkws = {'alpha':0.9}
fillkws = {'alpha':0.3}

ax[0].set_title('Total runoff, all glaciers')
ax[0].fill_between(B.t, B.runoff_all*fact_m3_day, 
           color = col.col('g'), **fillkws)
ax[0].plot(B.t, B.runoff_all*fact_m3_day, 
           color = col.col('g'), **plkws)
ax[0].set_ylabel('[m$^3$ s$^{-1}$]')

ax[1].fill_between(B.t, B.runoff_hb*fact_m3_day,
           color = col.col('o'), **fillkws, label = 'Hartogbukta')
ax[1].plot(B.t, B.runoff_hb*fact_m3_day, color = col.col('o'), **plkws)
ax[1].fill_between(B.t, B.runoff_hb*fact_m3_day, (B.runoff_hb+B.runoff_ss)*fact_m3_day,
           color = col.col('b'), **fillkws, label = 'Storisstr')
ax[1].plot(B.t, (B.runoff_hb+B.runoff_ss)*fact_m3_day, color = col.col('k'), label = 'SS+HB', **plkws)
ax[1].set_title('Runoff, Storisstraumen area')
ax[1].set_ylabel('[m$^3$ s$^{-1}$]')
ax[1].legend(loc = 2, handlelength = 1, fontsize = 10)

W = 100  
ax[2].fill_between(B.t, B.runoff_hb*fact_m3_day/W,
           color = col.col('o'), **fillkws)
ax[2].plot(B.t, B.runoff_hb*fact_m3_day/W, label = 'Hartogbukta', color = col.col('o'), **plkws)
ax[2].fill_between(B.t, B.runoff_ss*fact_m3_day/W,
           color = col.col('b'), **fillkws)
ax[2].plot(B.t, B.runoff_ss*fact_m3_day/W, color = col.col('b'), label = 'Storisstr N', **plkws)
ax[2].set_title('Runoff per width with W=%.0f m'%W)
ax[2].set_ylabel('[m$^2$ s$^{-1}$]')
ax[2].legend(loc = 2, handlelength = 1, fontsize = 10)
#.yscale('log')


plt.tight_layout()


for axn in ax:
    pretty.axthick(axn)
    pretty.lrbars(axn)
    axn.grid(alpha = 0.2)
    pretty.ticklabelsize(axn, 10)
    
pl.ym_xlab(axn, ymonths = [1, 7], fmt = '%b %d\n%Y',)


In [22]:
figdir = '/home/oyvindl/work/projects/barents_glaciers/glacierocean-austfonna/data/runoff/figures/'
fig.savefig(figdir + 'runoff_time_series_quick.png', bbox_inches = 'tight')

In [24]:
D = Bunch()
D.t = B.t

D.runoff_hartogbukta  = B.runoff_hb*fact_m3_day
D.runoff_storisstraumen  = B.runoff_ss*fact_m3_day
D.runoff_sshb  = B.runoff_hbss*fact_m3_day
D.runoff_all  = B.runoff_all*fact_m3_day

D.lon_hb = A.Longitude[nns_hb].to_numpy()
D.lat_hb = A.Latitude[nns_hb].to_numpy()
D.lon_ss = A.Longitude[nns_ss].to_numpy()
D.lat_ss = A.Latitude[nns_ss].to_numpy()

D.units = 'm3/second'
D.source_script = 'Read updated runoff data.ipynb'
D.source_data = fns
D.meta = '''
Runoff from Louise. Start 1.1.2014, end 4.6.2022.

Calculated for runoff points at ss and hb.
'''

pickle_save(D, 'runoff_storisstraumen_2014_2022.p')